In [44]:
#r "nuget: GraphQL.Client"
#r "nuget: GraphQL.Client.Serializer.Newtonsoft"
#r "nuget: Plotly.NET.Interactive,  2.0.0"

open GraphQL
open GraphQL.Client.Http
open GraphQL.Client.Serializer.Newtonsoft
open System
open Plotly.NET

//let baseUrl = "https://camaradosdeputadosgql-qboe6n5gda-uc.a.run.app/graphql"
let baseUrl = "http://localhost:5078/graphql"
let graphQLClient = new GraphQLHttpClient(baseUrl, new NewtonsoftJsonSerializer());

let deputyRequest party state year month = new GraphQLRequest (
    $"""
    query {{
        deputies (party: "{party}" state: "{state}") {{
            name
            party
            picture
            expenses (year: "{year}" month: "{month}") {{
                netValue
                documentDate
            }}
        }}
    }}
    """
)

let expenseRequest name year = new GraphQLRequest (
    $"""
    query {{
        deputies (name: "{name}") {{
            name
            party
            state
            picture
            expenses (year: "{year}") {{
                netValue
                documentDate
                supplierCnpjOrCpf
                supplierName
            }}
        }}
    }}
    """
)

type DeputyResponse = {
    deputies: Deputy list
} 
and Deputy = {
    name: string
    party: string
    picture: string
    expenses: Expense[]
}
and Expense = {
    netValue: decimal
    supplierName: string
    documentDate: Nullable<DateTime>
}

let GetDeputies party state year month =
    let response =
        graphQLClient.SendQueryAsync<DeputyResponse>(deputyRequest party state year month)
        |> Async.AwaitTask
        |> Async.RunSynchronously
    response.Data.deputies

let GetExpenses deputy year = 
    let response = 
        graphQLClient.SendQueryAsync<DeputyResponse>(expenseRequest deputy year)
        |> Async.AwaitTask
        |> Async.RunSynchronously
    response.Data.deputies |> Seq.head

let scatter (deputy:Deputy) =
    let dates, values =
        if deputy.expenses <> null then
            deputy.expenses
            |> Array.filter(fun expense -> expense.documentDate.HasValue)
            |> Array.sortBy(fun expense -> expense.documentDate.Value)
            |> Array.map (fun expense -> expense.documentDate.Value, expense.netValue)
            |> Array.unzip
        else
            [||],[||]
    Chart.Line(dates, values)
    |> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
    |> Chart.withTraceInfo deputy.name

let expenseChart (deputy:Deputy) =
    let labels, values =
        deputy.expenses
        |> Seq.groupBy(fun expense -> expense.supplierName)
        |> Seq.map (fun (name, expenses) ->
            let total =
                expenses
                |> Seq.sumBy ( fun expense -> expense.netValue )
            name, total
        )
        |> Seq.toList
        |> List.unzip
    Chart.Pie(Labels=labels, values=values)
    |> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
    

Installed Packages GraphQL.Client, 5.1.0 GraphQL.Client.Serializer.Newtonsoft, 5.1.0 Plotly.NET.Interactive, 2.0.0

In [45]:
let search = "Eduardo Bolsonaro"
let deputado = GetExpenses search 2022 
deputado
|> expenseChart
|> Chart.withTitle $"Despesas por fornecedor - {search} - 2022"

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [42]:
seq { yield deputado }
|> Seq.map scatter
|> Chart.combine

Error: input.fsx (2,12)-(2,19) typecheck error Type mismatch. Expecting a
    'FSI_0043.Deputy -> Plotly.NET.GenericChart.GenericChart'    
but given a
    'FSI_0063.Deputy -> Plotly.NET.GenericChart.GenericChart'    
The type 'FSI_0043.Deputy' does not match the type 'FSI_0063.Deputy'

In [43]:
HTML($"<img src=\"{deputado.picture}\"></img>")